In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.5.0


In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(4):
            self.model.add(dense(512, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

In [7]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(4) :
        model.add(dense(512, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_deep'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [10]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.6533, train_loss: 2.03881884, train_accuracy: 0.4688, test_Accuracy: 0.2706
Epoch: [ 0] [    1/  468] time: 0.9077, train_loss: 1.87532616, train_accuracy: 0.5234, test_Accuracy: 0.4161
Epoch: [ 0] [    2/  468] time: 1.1770, train_loss: 1.68615568, train_accuracy: 0.5469, test_Accuracy: 0.4993
Epoch: [ 0] [    3/  468] time: 1.4727, train_loss: 1.41487789, train_accuracy: 0.6641, test_Accuracy: 0.5969
Epoch: [ 0] [    4/  468] time: 1.7479, train_loss: 1.16061139, train_accuracy: 0.7031, test_Accuracy: 0.6425
Epoch: [ 0] [    5/  468] time: 2.0222, train_loss: 0.87822056, train_accuracy: 0.7734, test_Accuracy: 0.6834
Epoch: [ 0] [    6/  468] time: 2.2576, train_loss: 0.79570675, train_accuracy: 0.7656, test_Accuracy: 0.7343
Epoch: [ 0] [    7/  468] time: 2.5111, train_loss: 0.64266080, train_accuracy: 0.8281, test_Accuracy: 0.7853
Epoch: [ 0] [    8/  468] time: 2.7734,

Epoch: [ 0] [   74/  468] time: 21.2702, train_loss: 0.21500003, train_accuracy: 0.9219, test_Accuracy: 0.9133
Epoch: [ 0] [   75/  468] time: 21.6233, train_loss: 0.21381852, train_accuracy: 0.9375, test_Accuracy: 0.9073
Epoch: [ 0] [   76/  468] time: 21.9833, train_loss: 0.28186750, train_accuracy: 0.9219, test_Accuracy: 0.9187
Epoch: [ 0] [   77/  468] time: 22.3992, train_loss: 0.21756881, train_accuracy: 0.9297, test_Accuracy: 0.9241
Epoch: [ 0] [   78/  468] time: 22.7433, train_loss: 0.17336610, train_accuracy: 0.9297, test_Accuracy: 0.9161
Epoch: [ 0] [   79/  468] time: 23.0724, train_loss: 0.23117134, train_accuracy: 0.9297, test_Accuracy: 0.9114
Epoch: [ 0] [   80/  468] time: 23.4165, train_loss: 0.31219590, train_accuracy: 0.9219, test_Accuracy: 0.9156
Epoch: [ 0] [   81/  468] time: 23.7686, train_loss: 0.13986924, train_accuracy: 0.9609, test_Accuracy: 0.9201
Epoch: [ 0] [   82/  468] time: 24.1685, train_loss: 0.26093042, train_accuracy: 0.9297, test_Accuracy: 0.9244
E

Epoch: [ 0] [  148/  468] time: 45.8478, train_loss: 0.13707542, train_accuracy: 0.9531, test_Accuracy: 0.9480
Epoch: [ 0] [  149/  468] time: 46.1499, train_loss: 0.08622555, train_accuracy: 0.9844, test_Accuracy: 0.9440
Epoch: [ 0] [  150/  468] time: 46.4671, train_loss: 0.11896791, train_accuracy: 0.9688, test_Accuracy: 0.9342
Epoch: [ 0] [  151/  468] time: 46.7613, train_loss: 0.24503204, train_accuracy: 0.9375, test_Accuracy: 0.9355
Epoch: [ 0] [  152/  468] time: 47.0705, train_loss: 0.10503571, train_accuracy: 0.9609, test_Accuracy: 0.9423
Epoch: [ 0] [  153/  468] time: 47.3757, train_loss: 0.09857136, train_accuracy: 0.9766, test_Accuracy: 0.9431
Epoch: [ 0] [  154/  468] time: 47.7038, train_loss: 0.11023678, train_accuracy: 0.9609, test_Accuracy: 0.9405
Epoch: [ 0] [  155/  468] time: 48.0010, train_loss: 0.07440609, train_accuracy: 0.9766, test_Accuracy: 0.9323
Epoch: [ 0] [  156/  468] time: 48.3137, train_loss: 0.19423658, train_accuracy: 0.9297, test_Accuracy: 0.9274
E

Epoch: [ 0] [  222/  468] time: 69.9523, train_loss: 0.16638228, train_accuracy: 0.9688, test_Accuracy: 0.9469
Epoch: [ 0] [  223/  468] time: 70.2684, train_loss: 0.14655086, train_accuracy: 0.9688, test_Accuracy: 0.9493
Epoch: [ 0] [  224/  468] time: 70.5716, train_loss: 0.11748511, train_accuracy: 0.9688, test_Accuracy: 0.9529
Epoch: [ 0] [  225/  468] time: 70.8788, train_loss: 0.03364221, train_accuracy: 0.9922, test_Accuracy: 0.9533
Epoch: [ 0] [  226/  468] time: 71.1680, train_loss: 0.09427172, train_accuracy: 0.9844, test_Accuracy: 0.9521
Epoch: [ 0] [  227/  468] time: 71.4620, train_loss: 0.10807845, train_accuracy: 0.9609, test_Accuracy: 0.9491
Epoch: [ 0] [  228/  468] time: 71.7543, train_loss: 0.25033662, train_accuracy: 0.8906, test_Accuracy: 0.9473
Epoch: [ 0] [  229/  468] time: 72.0854, train_loss: 0.12486541, train_accuracy: 0.9609, test_Accuracy: 0.9486
Epoch: [ 0] [  230/  468] time: 72.4075, train_loss: 0.21253058, train_accuracy: 0.9375, test_Accuracy: 0.9507
E

Epoch: [ 0] [  296/  468] time: 93.4527, train_loss: 0.07752925, train_accuracy: 0.9844, test_Accuracy: 0.9545
Epoch: [ 0] [  297/  468] time: 93.7509, train_loss: 0.07070548, train_accuracy: 0.9844, test_Accuracy: 0.9521
Epoch: [ 0] [  298/  468] time: 94.0471, train_loss: 0.07822473, train_accuracy: 0.9609, test_Accuracy: 0.9539
Epoch: [ 0] [  299/  468] time: 94.3532, train_loss: 0.03984796, train_accuracy: 0.9844, test_Accuracy: 0.9571
Epoch: [ 0] [  300/  468] time: 94.6415, train_loss: 0.06216808, train_accuracy: 0.9766, test_Accuracy: 0.9594
Epoch: [ 0] [  301/  468] time: 94.9467, train_loss: 0.16061279, train_accuracy: 0.9531, test_Accuracy: 0.9578
Epoch: [ 0] [  302/  468] time: 95.2393, train_loss: 0.06093615, train_accuracy: 0.9922, test_Accuracy: 0.9560
Epoch: [ 0] [  303/  468] time: 95.5315, train_loss: 0.08590095, train_accuracy: 0.9688, test_Accuracy: 0.9560
Epoch: [ 0] [  304/  468] time: 95.8427, train_loss: 0.07114425, train_accuracy: 0.9766, test_Accuracy: 0.9564
E

Epoch: [ 0] [  370/  468] time: 116.2610, train_loss: 0.14886448, train_accuracy: 0.9609, test_Accuracy: 0.9616
Epoch: [ 0] [  371/  468] time: 116.5582, train_loss: 0.07843255, train_accuracy: 0.9688, test_Accuracy: 0.9626
Epoch: [ 0] [  372/  468] time: 116.8534, train_loss: 0.06789203, train_accuracy: 0.9844, test_Accuracy: 0.9611
Epoch: [ 0] [  373/  468] time: 117.1525, train_loss: 0.05129852, train_accuracy: 0.9844, test_Accuracy: 0.9615
Epoch: [ 0] [  374/  468] time: 117.4607, train_loss: 0.11730521, train_accuracy: 0.9766, test_Accuracy: 0.9632
Epoch: [ 0] [  375/  468] time: 117.7559, train_loss: 0.09265608, train_accuracy: 0.9922, test_Accuracy: 0.9656
Epoch: [ 0] [  376/  468] time: 118.0671, train_loss: 0.05869444, train_accuracy: 0.9766, test_Accuracy: 0.9646
Epoch: [ 0] [  377/  468] time: 118.3857, train_loss: 0.12628064, train_accuracy: 0.9766, test_Accuracy: 0.9635
Epoch: [ 0] [  378/  468] time: 118.6939, train_loss: 0.15615386, train_accuracy: 0.9531, test_Accuracy:

Epoch: [ 0] [  444/  468] time: 141.1897, train_loss: 0.07922908, train_accuracy: 0.9609, test_Accuracy: 0.9626
Epoch: [ 0] [  445/  468] time: 141.5258, train_loss: 0.08093455, train_accuracy: 0.9766, test_Accuracy: 0.9632
Epoch: [ 0] [  446/  468] time: 141.8639, train_loss: 0.10779739, train_accuracy: 0.9609, test_Accuracy: 0.9666
Epoch: [ 0] [  447/  468] time: 142.2130, train_loss: 0.10351336, train_accuracy: 0.9688, test_Accuracy: 0.9663
Epoch: [ 0] [  448/  468] time: 142.5870, train_loss: 0.08177997, train_accuracy: 0.9688, test_Accuracy: 0.9660
Epoch: [ 0] [  449/  468] time: 142.9211, train_loss: 0.07363286, train_accuracy: 0.9844, test_Accuracy: 0.9631
Epoch: [ 0] [  450/  468] time: 143.2602, train_loss: 0.05182850, train_accuracy: 0.9844, test_Accuracy: 0.9604
Epoch: [ 0] [  451/  468] time: 143.6002, train_loss: 0.03366864, train_accuracy: 0.9922, test_Accuracy: 0.9584
Epoch: [ 0] [  452/  468] time: 143.9314, train_loss: 0.13549654, train_accuracy: 0.9609, test_Accuracy: